# Point observations

In order to skill assess our model, we need observational data. The data may come from a file or web api. We will cover the following situations here: 

* File
    - dfs0
    - csv/excel
    - NetCDF
* REST API

FMskill has the class [PointObservation](https://dhi.github.io/fmskill/api.html#fmskill.observation.PointObservation) for working with point observations. 

Point observations consist of time-value-pairs (data) and *meta* data such as

* data type (e.g. water level)
* unit (e.g. meter)
* position (coordinates + coordinate reference system)

FMskill is agnostic to the coordinate reference system (CRS) and it is therefore the responsibility of the user to make sure that all data (observations and model) use the same CRS.

In [ ]:
from fmskill import PointObservation
from mikeio import eum

## Observations from dfs0

Dfs0 files are obviously a very common container format for point observation data at DHI. Besides the data (time-value-pairs), it typically contains meta data information about the data type (EUM type, e.g. water level) and data unit (EUMUnit, e.g. meter). It can potentially store geographical information too, but this is rarely the case. We typically need to provide this information ourselves.

In [ ]:
fn = 'data/SW/HKNA_Hm0.dfs0'
pos = (4.2420, 52.6887)   # LONG/LAT

## The PointObservation class

The PointObservation class can be instantiated with a dfs0 file name, the item, the position and optionally also a user-defined name.

PointObservation has basic properties like start_time, end_time, n_points, name, etc

PointObservation has two simple plot methods:

* plot() - a time series plot
* hist() - a histogram

Note that the PointObservation object takes a single item only. 

In [ ]:
o1 = PointObservation(fn, item=0, x=pos[0], y=pos[1], name="HKNA_from_dfs0")
o1

In [ ]:
o1.start_time, o1.end_time

In [ ]:
o1.n_points

In [ ]:
o1.df.head()

In [ ]:
o1.itemInfo

In [ ]:
o1.hist();

In [ ]:
o1.plot(figsize=(12,6));

## Observations from csv/excel

Pandas is our friend. FMSkill PointObservation can be initialized with either a dfs0 or a pandas DataFrame. Hence, for other file types than dfs0 the workflow is to first create a DataFrame with the data e.g. using pd.read_csv() or pd.read_excel(). 

You need to provide the **position** and preferably also **EUM info** (for nice plotting). 

In [ ]:
import pandas as pd
df = pd.read_csv('data/SW/HKNA_Hm0.csv', index_col=0, parse_dates=True)   # make sure index is DateTimeIndex!
#df = pd.read_excel('data/SW/HKNA_Hm0.xlsx', index_col=0, parse_dates=True)
df.head()

In [ ]:
df.plot();

In [ ]:
o2 = PointObservation(df, item="Hm0", x=pos[0], y=pos[1], name='Hm0_from_csv')

set the itemInfo for nicer plotting, and EUM validation when matching with a model result

In [ ]:
o2.itemInfo = eum.ItemInfo(eum.EUMType.Significant_wave_height)

In [ ]:
o2.plot();

## Observations from NetCDF

A NetCDF file is best handled with xarray. It often contains meta data that you can use for constructing your point observation object.

In [ ]:
import xarray as xr
ds = xr.open_dataset('data/SW/HKNA_Hm0.nc')
ds

In [ ]:
ds.position

In [ ]:
ds.Hm0

In [ ]:
ds.Hm0.plot();

In [ ]:
df = ds.to_dataframe()
o3 = PointObservation(df, item="Hm0", x=ds.position[0], y=ds.position[1], name='Hm0_from_nc')

In [ ]:
o3.itemInfo = eum.ItemInfo(eum.EUMType.Significant_wave_height)

In [ ]:
o3.plot();

## Observations from web APIs

FMSkill has functionality for accessing observational data trough a few selected APIs - that functionality will soon moved to a separate repo WatObs. More about that later.

For now - check out the [DMI_observations.ipynb](https://nbviewer.jupyter.org/github/DHI/fmskill/blob/main/notebooks/DMI_observations.ipynb) notebook. 